In [ ]:
# hello this is 심상균
### run all 돌리지 마시오!!! 중간에 실행해서는 안되는 코드 존재!!! ###

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense
from tensorflow.keras.utils import plot_model
import pydot
import graphviz

In [ ]:
# 임의의 시계열 데이터 생성 (사인파)
def generate_sin_wave(seq_length, num_sequences):
    # linspace : 0과 100을 seq_length * num_sequences 개수만큼 쪼개서 리스트로 만들어 버렷! >_<
    x = np.linspace(0, 100, seq_length * num_sequences)
    y = np.sin(x)
    # reshape : 개념은 좀 어려울 수 있으니 모르면 걍 검색하셈 ㅇㅇ,, 대충 num_sequences 차원으로 하나 나누고 seq_length 차원으로 한번 더 나눈거라 생각하셈
    return y.reshape(num_sequences, seq_length, 1)

seq_length = 50  # 시퀀스 길이
num_sequences = 1000  # 시퀀스 개수
data = generate_sin_wave(seq_length, num_sequences) # 데이터 생성하기!!

# 데이터 그래프 그리기
x = np.linspace(0, 100, seq_length * num_sequences)
y = np.sin(x)
plt.plot(x, y)
plt.show()

In [ ]:
# 데이터셋 분리
split = int(0.8 * num_sequences)
train_data = data[:split]
test_data = data[split:]

In [ ]:
print(data)

In [ ]:
# 모델 구축
# Sequential : Keras 에서 제공하는 신경망 모델 클래스임 간단하게 레이어를 순차적으로 쌓아서 신경망을 구성할때 사용함
model = Sequential()

# 전체 LSTM 퍼셉트론 개수는 50개야~~ 그리고 내부의 활성화 함수는 뤨루임~~  타임스텝은 시퀀스 길이만큼이야 특징은 하나밖에 없지
# 그러면 그시기 그시기 해서 잘 뿅뿅 해서 모델이 만들어져 
model.add(LSTM(50, activation='relu', input_shape=(seq_length, 1)))
# 50개의 퍼셉트론에서의 출력값을 하나로 짬뽕해서 최종 출력은 임의의 실후가 될거야~
model.add(Dense(1))
# 모델은 아담아담으로 최적화하고 평균제곱오차로 손실값 정할거야
model.compile(optimizer='adam', loss='mse')

In [ ]:
'''
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
이 코드는 실행시키지 말 것!!!!!!!
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
#########################################################################
'''
# 모델 구축 예시
# return_sequences = True를 한 경우에는 각 time step별 hidden state들이 출력된다.
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from numpy import array

# 타임스텝이 3, 특징이 1개인 인풋 그시기 생성
# 특징의 개수는 주가, 거래량, 온도 등의 다양한 정보를 한번에 입력할때 1보다 커짐
# Input 은 입력 데이터의 모양을 정함
inputs1 = Input(shape=(3, 1))
lstm1 = LSTM(3, return_sequences=False)(inputs1)
model = Model(inputs=inputs1, outputs=lstm1)

data = array([0.1, 0.2, 0.3]).reshape((1,3,1))

print(data)
print(model.predict(data))

In [ ]:
# 모델 학습
X_train = train_data[:, :-1]
y_train = train_data[:, -1]
model.fit(X_train, y_train, epochs=20, verbose=1)

In [ ]:
X_test = test_data[:, :-1]
y_test = test_data[:, -1]
predictions = model.predict(X_test)

In [ ]:
plt.plot(y_test, label='True')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.show()

In [ ]:
model.save('mnist_model.keras')

In [ ]:
# 모델 불러오기
loaded_model = tf.keras.models.load_model('./mnist_model.keras')

# 모델 아키텍처 확인
loaded_model.summary()

# 모델 컴파일 정보 확인
print("Optimizer:", loaded_model.optimizer)
print("Loss function:", loaded_model.loss)
print("Metrics:", loaded_model.metrics_names)

# 모델의 가중치 확인
weights = loaded_model.get_weights()
print("Model weights:", weights)

# 불러온 모델을 사용하여 예측
predictions = loaded_model.predict(X_test)

# 예측 결과 확인 (예: 첫 번째 테스트 이미지의 예측 결과)
print(predictions[0])  # softmax 출력
print(np.argmax(predictions[0]))  # 예측된 클래스